데이터에 매개변수를 최적화하여 모델을 학습하고, 검증하고 테스트할 차례! <br>
파라미터들을 최적화(Optimize)

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

# 하이퍼파라미터

하이퍼파라미터는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수
학습 시에는 다음과 같은 하이퍼파라미터를 정의 <br>
* epoch: 데이터셋을 반복하는 횟수 <br>
* batch size: 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수<br>
* learning rate: 각 배치/에폭에서 모델의 매개변수를 조절하는 비율

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5


# 최적화 단계 (Optimization Loop)

* Train Loop: 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴
* Validation/Test Loop: 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복

# 손실함수

In [3]:
# 손실함수를 초기화
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 (Optimizer)

최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 배개변수를 조정하는 과정. 최적화 알고리즘은 이 과정이 수행되는 방식을 정의

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

😍😍Train에서 최적화는 세 단계로 이루어짐:
* optimizer.zero_grad(): 모델 매개변수의 변화도를 재설정, 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정
* loss.backward(): 예측 손실(prediction loss)을 역전파. Pytorch는 각 매개변수에 대한 손실의 변화도를 저장
* optimizer.step(): 역전파 단계에서 수집한 변화도로 매개변수를 조정


In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print('loss : {}, {}: {}'.format(loss, current, size))
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 10
for epoch in range(epochs):
    print('=====Epoch {}=====\n'.format(epoch))
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('Done!!')
    

=====Epoch 0=====

loss : 2.173855781555176, 64: 60000
loss : 2.1565022468566895, 6464: 60000
loss : 2.0992095470428467, 12864: 60000
loss : 2.1105430126190186, 19264: 60000
loss : 2.064605236053467, 25664: 60000
loss : 2.012636184692383, 32064: 60000
loss : 2.036026954650879, 38464: 60000
loss : 1.967970371246338, 44864: 60000
loss : 1.9683165550231934, 51264: 60000
loss : 1.886704683303833, 57664: 60000
Test Error: 
 Accuracy: 57.6%, Avg loss: 1.889095 

=====Epoch 1=====

loss : 1.9296544790267944, 64: 60000
loss : 1.8882620334625244, 6464: 60000
loss : 1.7741279602050781, 12864: 60000
loss : 1.8087595701217651, 19264: 60000
loss : 1.6998146772384644, 25664: 60000
loss : 1.6586521863937378, 32064: 60000
loss : 1.6765905618667603, 38464: 60000
loss : 1.5891015529632568, 44864: 60000
loss : 1.60635507106781, 51264: 60000
loss : 1.4916495084762573, 57664: 60000
Test Error: 
 Accuracy: 60.9%, Avg loss: 1.517055 

=====Epoch 2=====

loss : 1.589207649230957, 64: 60000
loss : 1.5469381809